In [ ]:
# Attempt at T5 finetuning with simplet5

In [9]:
# import
from simplet5 import SimpleT5
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
# import dataset from csv
congress_115_text_df = pd.read_csv('congress_115_text.csv')
congress_115_text_df.head()

In [7]:
# format dataset for simpleT5, ie bills stored as "source_text" beginning with "summarize: " 
# and summarizations stored as "target_text"
text_to_summarize_df = congress_115_text_df.rename(columns={"Summary":"target_text", "Text":"source_text"})
text_to_summarize_df = text_to_summarize_df[['source_text', 'target_text']]
text_to_summarize_df.head()

,source_text,target_text
0,<html><body><pre>\n[Congressional Bills 115th ...,(This measure has not been amended since it wa...
1,<html><body><pre>\n[Congressional Bills 115th ...,(This measure has not been amended since it wa...
2,<html><body><pre>\n[Congressional Bills 115th ...,Financial CHOICE Act of 2017\n\n (Sec. 2) This...
3,<html><body><pre>\n[Congressional Bills 115th ...,Raise the Wage Act\n\nThis bill amends the Fai...
4,<html><body><pre>\n[Congressional Bills 115th ...,Smithsonian Women's History Museum Act\n\nThis...


In [8]:
# T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
text_to_summarize_df['source_text'] = "summarize: " + text_to_summarize_df['source_text']
text_to_summarize_df.head()

,source_text,target_text
0,summarize: <html><body><pre>\n[Congressional B...,(This measure has not been amended since it wa...
1,summarize: <html><body><pre>\n[Congressional B...,(This measure has not been amended since it wa...
2,summarize: <html><body><pre>\n[Congressional B...,Financial CHOICE Act of 2017\n\n (Sec. 2) This...
3,summarize: <html><body><pre>\n[Congressional B...,Raise the Wage Act\n\nThis bill amends the Fai...
4,summarize: <html><body><pre>\n[Congressional B...,Smithsonian Women's History Museum Act\n\nThis...


In [11]:
train_df, test_df = train_test_split(text_to_summarize_df, test_size=0.2)
train_df.shape, test_df.shape

((1804, 2), (451, 2))

In [12]:
# instantiate
model = SimpleT5()

# load (supports t5, mt5, byT5 models)
model.from_pretrained("t5","t5-base")

# train
model.train(train_df=train_df, # pandas dataframe with 2 columns: source_text & target_text
            eval_df=eval_df, # pandas dataframe with 2 columns: source_text & target_text
            source_max_token_len = 512, 
            target_max_token_len = 128,
            batch_size = 8,
            max_epochs = 5,
            use_gpu = True,
            outputdir = "outputs",
            early_stopping_patience_epochs = 0,
            precision = 32
            )

# load trained T5 model
model.load_model("t5","path/to/trained/model/directory", use_gpu=False)

# predict
model.predict("input text for prediction")

NameError: name 'eval_df' is not defined

In [14]:
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
model.train(train_df=train_df[:5000],
            eval_df=test_df[:100], 
            source_max_token_len=128, 
            target_max_token_len=50, 
            batch_size=8, max_epochs=3, use_gpu=False)
     

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Missing logger folder: /Users/jim/ShaolinAI_Bootcamp/ShaolinAI-Project-3/lightning_logs

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/Users/jim/anaconda3/envs/mlenv/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Global seed set to 42
/Users/jim/anaconda3/envs/mlenv/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Training: 0it [00:00, ?it/s]

[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
2023-11-21 16:06:50.266395: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/jim/anaconda3/envs/mlenv/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
